### Things to try: (for tree-based model)
- item_price: decimal part, item_price_bin
- categorical variable interation: city&item_category, city&item_price_bin, item_category&item_price_bin
- lag features
- mean encoding
- knn features
- prophet forecasting
- word embedding

### References:
- BOW: http://scikit-learn.org/stable/modules/feature_extraction.html#vectorizing-a-large-text-corpus-with-the-hashing-trick
- Word2vec: http://mccormickml.com/2016/04/12/googles-pretrained-word2vec-model-in-python/
- List of Pretrained Word Embeddings: http://ahogrammer.com/2017/01/20/the-list-of-pretrained-word-embeddings/

### Feature extration on text: item and item_cats (using word2vec)

In [2]:
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import lightgbm as lgb
from catboost import CatBoostRegressor
from tqdm import tqdm_notebook
from itertools import product

In [3]:
import feather
import pyarrow
hol = pyarrow.feather.read_feather('hol.feather')
it_en = pyarrow.feather.read_feather('it_en.feather')
itc_en = pyarrow.feather.read_feather('itc_en.feather')
sh_en = pyarrow.feather.read_feather('sh_en.feather')
coordinates = pd.read_csv('coordinates.csv')
city_name_eng = pd.read_csv('city_name_eng.csv')

In [4]:
import re
import pandas as pd
import nltk.data
from nltk.corpus import stopwords  # if not installed, do nltk.download('stopwords')
def sentence_to_wordlist (sentence, remove_stopwords=False ):

    review_text = re.sub("[^a-zA-Z]"," ", sentence)

        # 3. Convert words to lower case and split them
    words = review_text.lower().split()

        # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]

        # 5. Return a list of words
    return(words)

In [9]:
import gensim
from gensim.models import Word2Vec
from gensim.models import word2vec
from gensim.models import Phrases
import logging

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [10]:
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin.gz', binary=True)  

In [97]:
it_en_word_embed_mean = []
it_en_word_embed_sum = []
for en_it_name in it_en.iloc[:,3]:
    words = sentence_to_wordlist(en_it_name,True)
    if any(word in model.vocab for word in words):
        words_vec_mean = np.mean(np.vstack([model[word] for word in words if word in model.vocab]),axis=0)
        words_vec_sum = np.sum(np.vstack([model[word] for word in words if word in model.vocab]),axis=0)
    else: 
        words_vec_mean = np.zeros((300,))
        words_vec_sum = np.zeros((300,))
    it_en_word_embed_mean.append(words_vec_mean)
    it_en_word_embed_sum.append(words_vec_sum)

In [98]:
index_cols1 = ['it_en_word_embed_mean' + str(a) for a in range(1,301)]
index_cols2 = ['it_en_word_embed_sum' + str(a) for a in range(1,301)]
it_en_word_embed_mean = pd.DataFrame(np.vstack(it_en_word_embed_mean), columns = index_cols1,dtype=np.float32)
it_en_word_embed_sum = pd.DataFrame(np.vstack(it_en_word_embed_sum), columns = index_cols2,dtype=np.float32)

In [99]:
itc_en_word_embed_mean = []
itc_en_word_embed_sum = []
for en_itc_name in itc_en.iloc[:,2]:
    words = sentence_to_wordlist(en_itc_name,True)
    if any(word in model.vocab for word in words):
        words_vec_mean = np.mean(np.vstack([model[word] for word in words if word in model.vocab]),axis=0)
        words_vec_sum = np.sum(np.vstack([model[word] for word in words if word in model.vocab]),axis=0)
    else: 
        words_vec_mean = np.zeros((300,))
        words_vec_sum = np.zeros((300,))
    itc_en_word_embed_mean.append(words_vec_mean)
    itc_en_word_embed_sum.append(words_vec_sum)

In [100]:
index_cols3 = ['itc_en_word_embed_mean' + str(a) for a in range(1,301)]
index_cols4 = ['itc_en_word_embed_sum' + str(a) for a in range(1,301)]
itc_en_word_embed_mean = pd.DataFrame(np.vstack(itc_en_word_embed_mean), columns = index_cols3,dtype=np.float32)
itc_en_word_embed_sum = pd.DataFrame(np.vstack(itc_en_word_embed_sum), columns = index_cols4,dtype=np.float32)

In [111]:
# add word embedding features to raw tables
it_en_with_word_embed = pd.concat([it_en.reset_index(drop=True),it_en_word_embed_mean.reset_index(drop=True),it_en_word_embed_sum.reset_index(drop=True)],axis=1)
itc_en_with_word_embed = pd.concat([itc_en.reset_index(drop=True),itc_en_word_embed_mean.reset_index(drop=True),itc_en_word_embed_sum.reset_index(drop=True)],axis=1)

In [117]:
# it_en_with_word_embed.to_csv('it_en_with_word_embed.csv',index=False) may cause problem in first column (Russian)
# itc_en_with_word_embed.to_csv('itc_en_with_word_embed.csv',index=False) may cause problem in first column (Russian)
pyarrow.feather.write_feather(it_en_with_word_embed,'it_en_with_word_embed.feather')
pyarrow.feather.write_feather(itc_en_with_word_embed,'itc_en_with_word_embed.feather')